<a href="https://colab.research.google.com/github/tannyyaawadhwani-design/puthony/blob/main/eppproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ================================
# Employee Performance Dashboard (Interactive)
# Author: Tanya
# ================================
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# 1) Make realistic sample data (12 employees, 4 departments, 6 months)
np.random.seed(42)

employees = [f"Emp{i:02d}" for i in range(1, 13)]
departments = ["IT", "HR", "Sales", "Marketing"]
months = pd.period_range("2025-03", periods=6, freq="M").astype(str)  # 6 months

rows = []
for emp in employees:
    dept = np.random.choice(departments)
    base_perf = np.random.randint(65, 90)     # employee baseline performance
    base_salary = np.random.randint(35000, 80001)
    for m in months:
        # add some monthly noise
        perf = np.clip(int(np.random.normal(base_perf, 8)), 50, 100)
        hours = int(np.random.normal(168, 15))  # avg ~168 hrs/month
        # tiny month-based salary jitter
        sal = int(base_salary + np.random.normal(0, 1000))
        rows.append([emp, dept, m, perf, hours, sal])

df = pd.DataFrame(rows, columns=["Employee", "Department", "Month", "Performance", "Hours Worked", "Salary"])

# 2) Quick KPIs
dept_perf = df.groupby("Department")["Performance"].mean().sort_values(ascending=False)
top_emp_idx = df.groupby("Employee")["Performance"].mean().idxmax()
top_emp_avg = df.groupby("Employee")["Performance"].mean().max()
cor_salary_perf = df["Salary"].corr(df["Performance"])

print("Average Performance by Department:")
print(dept_perf.round(2))
print("\nTop Employee (avg across months):", top_emp_idx, "→", round(top_emp_avg, 2))
print("Salary ↔ Performance Correlation:", round(cor_salary_perf, 3))

# 3) Chart 1 — Department-wise average performance (bar)
fig_bar = px.bar(
    dept_perf.reset_index(),
    x="Department", y="Performance",
    title="Average Performance by Department",
    text="Performance"
)
fig_bar.update_traces(texttemplate="%{text:.1f}", textposition="outside")
fig_bar.update_layout(yaxis_title="Performance (avg)", xaxis_title=None, uniformtext_minsize=10, uniformtext_mode="hide")

# 4) Chart 2 — Salary vs Performance (scatter, colored by dept, hover info)
fig_scatter = px.scatter(
    df, x="Performance", y="Salary", color="Department",
    hover_data=["Employee", "Month", "Hours Worked"],
    title="Salary vs Performance (Hover for details)",
)
fig_scatter.update_layout(xaxis_title="Performance", yaxis_title="Salary")

# Add a department dropdown to toggle visibility
buttons = []
for d in ["All"] + departments:
    visible = []
    # Plotly Express creates one trace per department in a color grouping
    # Build visibility mask per department
    for tr in fig_scatter.data:
        if d == "All":
            visible.append(True)
        else:
            # trace.name holds the department
            visible.append(tr.name == d)
    buttons.append(dict(label=d, method="update", args=[{"visible": visible}]))

fig_scatter.update_layout(
    updatemenus=[dict(
        type="dropdown",
        x=1.02, y=1,
        xanchor="left", yanchor="top",
        buttons=buttons,
        showactive=True
    )]
)

# 5) Chart 3 — Performance trend by department over months (lines)
dept_month = df.groupby(["Month", "Department"])["Performance"].mean().reset_index()
fig_line = px.line(
    dept_month, x="Month", y="Performance", color="Department",
    markers=True, title="Monthly Performance Trend by Department"
)
fig_line.update_layout(yaxis_title="Avg Performance", xaxis_title=None)

# 6) Chart 4 — Employee Utilization: Hours Worked vs Performance (bubble)
fig_bubble = px.scatter(
    df, x="Hours Worked", y="Performance", size="Salary", color="Department",
    hover_data=["Employee", "Month"],
    title="Utilization vs Performance (Bubble size = Salary)"
)
fig_bubble.update_layout(xaxis_title="Hours Worked", yaxis_title="Performance")

# Show all figures (one below another) in Colab
fig_bar.show()
fig_scatter.show()
fig_line.show()
fig_bubble.show()

# 7) Export a single-page HTML dashboard (includes all 4 charts)
from plotly.subplots import make_subplots

# Create a 2x2 dashboard layout
dashboard = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Avg Performance by Department",
        "Salary vs Performance",
        "Monthly Performance by Department",
        "Utilization vs Performance"
    )
)

# Add traces from each figure into the dashboard
for tr in fig_bar.data:
    dashboard.add_trace(tr, row=1, col=1)

for tr in fig_scatter.data:
    dashboard.add_trace(tr, row=1, col=2)

for tr in fig_line.data:
    dashboard.add_trace(tr, row=2, col=1)

for tr in fig_bubble.data:
    dashboard.add_trace(tr, row=2, col=2)

# Carry over the dropdown from the scatter chart (applies to all scatter traces)
dashboard.update_layout(
    title="Employee Performance & Productivity Dashboard (Interactive)",
    height=900,
    updatemenus=fig_scatter.layout.updatemenus  # reuse dept dropdown
)

# Axis labels
dashboard.update_xaxes(title_text="Department", row=1, col=1)
dashboard.update_yaxes(title_text="Avg Performance", row=1, col=1)
dashboard.update_xaxes(title_text="Performance", row=1, col=2)
dashboard.update_yaxes(title_text="Salary", row=1, col=2)
dashboard.update_xaxes(title_text="Month", row=2, col=1)
dashboard.update_yaxes(title_text="Avg Performance", row=2, col=1)
dashboard.update_xaxes(title_text="Hours Worked", row=2, col=2)
dashboard.update_yaxes(title_text="Performance", row=2, col=2)

# Save as HTML
dashboard.write_html("employee_performance_dashboard.html", include_plotlyjs="cdn")
print("\nSaved 'employee_performance_dashboard.html' in your Colab files. Download and share it!")

Average Performance by Department:
Department
HR           80.44
IT           80.00
Marketing    78.67
Sales        78.33
Name: Performance, dtype: float64

Top Employee (avg across months): Emp01 → 88.67
Salary ↔ Performance Correlation: -0.298



Saved 'employee_performance_dashboard.html' in your Colab files. Download and share it!
